In [ ]:
import os,sys
import kfp
import json

In [ ]:
components_url = "/mnt/dkube/pipeline/components/"
dkube_training_op = kfp.components.load_component_from_file(components_url + "training/component.yaml")
dkube_serving_op = kfp.components.load_component_from_file(components_url + "serving/component.yaml")
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(host=os.getenv("KF_PIPELINES_ENDPOINT"), existing_token=token, namespace=os.getenv("USERNAME"))

In [ ]:
with open("tuning.json", "r") as read_it:
     data = json.load(read_it)

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-mnist-pl',
    description='sample mnist pipeline with dkube components'
)
def mnist_pipeline(program='dkube-examples', dataset='mnist', model='mnist'):

    train       = dkube_training_op(container='{"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0"}',
                                    framework="tensorflow", version="2.0.0",
                                    program=str(program), run_script="python mnist/train.py",
                                    datasets=json.dumps([str(dataset)]), outputs=json.dumps([str(model)]),
                                    input_dataset_mounts='["/mnist"]',
                                    output_mounts='["/model"]',
                                    envs='[{"EPOCHS": "1"}]',
                                    tuning = json.dumps(data),
                                    auth_token=token)

    serving     = dkube_serving_op(model=train.outputs['artifact'], device='cpu', 
                                    serving_image='{"image":"ocdr/tensorflowserver:2.0.0"}',
                                    transformer_image='{"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0"}',
                                    transformer_project=str(program),
                                    transformer_code='mnist/transformer.py', auth_token=token).after(train)
    

In [ ]:
client.create_run_from_pipeline_func(mnist_pipeline, arguments={})

In [ ]:
#generate & upload pipeline (Optional)
import kfp.compiler as compiler
compiler.Compiler().compile(mnist_pipeline, "mnist-pipeline-tuning-demo.zip")
client.upload_pipeline("mnist-pipeline-tuning-demo.zip")